In [ ]:
import numpy as np

class SVM:
    def __init__(self, C=1.0, tol=1e-3, max_passes=5):
        self.C = C  # regularization parameter
        self.tol = tol  # tolerance for KKT conditions
        self.max_passes = max_passes  # maximum number of iterations without any alpha updates

    def fit(self, X, y):
        self.X = X
        self.y = y
        self.alpha = np.zeros(len(X))
        self.b = 0
        num_passes = 0

        while num_passes < self.max_passes:
            num_changed_alphas = 0
            for i in range(len(X)):
                Ei = self._predict(X[i]) - y[i]
                if (y[i] * Ei < -self.tol and self.alpha[i] < self.C) or (y[i] * Ei > self.tol and self.alpha[i] > 0):
                    j = np.random.choice(np.delete(np.arange(len(X)), i))  # Randomly select a second sample j != i
                    Ej = self._predict(X[j]) - y[j]
                    alpha_i_old, alpha_j_old = self.alpha[i], self.alpha[j]

                    # Compute L and H, the bounds on new alpha values
                    L, H = self._compute_L_H(self.alpha[j], self.alpha[i], y[j], y[i])

                    if L == H:
                        continue

                    # Compute eta (the similarity measure between the ith and jth input vectors)
                    eta = 2 * np.dot(X[i], X[j]) - np.dot(X[i], X[i]) - np.dot(X[j], X[j])

                    if eta >= 0:
                        continue

                    # Compute new alpha_j value using formula (12)
                    self.alpha[j] -= y[j] * (Ei - Ej) / eta

                    # Clip new alpha_j value
                    self.alpha[j] = max(L, min(H, self.alpha[j]))

                    # Check if alpha_j has changed significantly
                    if abs(self.alpha[j] - alpha_j_old) < 1e-5:
                        continue

                    # Update alpha_i using formula (11)
                    self.alpha[i] += y[i] * y[j] * (alpha_j_old - self.alpha[j])

                    # Compute the bias term b
                    b1 = self.b - Ei - y[i] * (self.alpha[i] - alpha_i_old) * np.dot(X[i], X[i]) - \
                         y[j] * (self.alpha[j] - alpha_j_old) * np.dot(X[i], X[j])

                    b2 = self.b - Ej - y[i] * (self.alpha[i] - alpha_i_old) * np.dot(X[i], X[j]) - \
                         y[j] * (self.alpha[j] - alpha_j_old) * np.dot(X[j], X[j])

                    if 0 < self.alpha[i] < self.C:
                        self.b = b1
                    elif 0 < self.alpha[j] < self.C:
                        self.b = b2
                    else:
                        self.b = (b1 + b2) / 2

                    num_changed_alphas += 1

            if num_changed_alphas == 0:
                num_passes += 1
            else:
                num_passes = 0

    def _predict(self, X):
        return np.dot(X, self.X.T).dot(self.alpha * self.y) + self.b

    def _compute_L_H(self, alpha_j, alpha_i, y_j, y_i):
        if y_i != y_j:
            return max(0, alpha_j - alpha_i), min(self.C, self.C - alpha_i + alpha_j)
        else:
            return max(0, alpha_i + alpha_j - self.C), min(self.C, alpha_i + alpha_j)

def train_svm_classifier(X_train, y_train):
    classifier = SVM(C=1.0)
    classifier.fit(X_train, y_train)
    return classifier